In [15]:
import pandas as pd
df = pd.read_csv("clean_tweet.csv")
#df.drop(['score','views','comments'],axis=1,inplace=True)
df.head(20)

,Unnamed: 0,text,postId
0,0,ve always been interested in machine learning ...,5
1,1,as researcher and instructor looking for open ...,7
2,2,not sure if this fits the scope of this se but...,9
3,3,one book that freely available is the elements...,10
4,4,am sure data science as will be discussed in t...,14
5,5,in which situations would one system be prefer...,15
6,6,use href to train data and predict clas,16
7,7,href rel nofollow libsvm is library for suppor,17
8,8,NaN,18
9,9,lots of people use the term em big data em in ...,19


In [16]:
df_p = pd.read_csv("import_code/csvs/tags_posts_rel.csv")
# df.drop(['external_author_id','language','publish_date','harvested_date'],axis=1,inplace=True)
df_p.head(20)

,:START_ID(Post),:END_ID(Tag)
0,5,machine-learning
1,7,education
2,7,open-source
3,14,data-mining
4,14,definitions
5,15,databases
6,16,machine-learning
7,16,bigdata
8,16,libsvm
9,19,bigdata


In [17]:
df_p.columns = ['postId', 'tag']

In [18]:
df_p.head()

,postId,tag
0,5,machine-learning
1,7,education
2,7,open-source
3,14,data-mining
4,14,definitions


In [19]:
my_df = pd.merge(df, df_p, on='postId')

In [20]:
my_df.head()

,Unnamed: 0,text,postId,tag
0,0,ve always been interested in machine learning ...,5,machine-learning
1,1,as researcher and instructor looking for open ...,7,education
2,1,as researcher and instructor looking for open ...,7,open-source
3,4,am sure data science as will be discussed in t...,14,data-mining
4,4,am sure data science as will be discussed in t...,14,definitions


In [21]:
for col in my_df.columns: 
    print(col) 

Unnamed: 0
text
postId
tag


In [22]:
my_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [26]:
my_df.groupby('tag').count()

,text,postId
tag,,
.net,6,6
3d-reconstruction,5,5
ab-test,26,26
accuracy,152,152
activation,1,1
...,...,...
word-embeddings,206,206
word2vec,227,227
xboost,1,1


In [17]:
my_df.head(20)

,text,postId,tag
0,ve always been interested in machine learning ...,5,machine-learning
1,as researcher and instructor looking for open ...,7,education
2,as researcher and instructor looking for open ...,7,open-source
3,am sure data science as will be discussed in t...,14,data-mining
4,am sure data science as will be discussed in t...,14,definitions
5,in which situations would one system be prefer...,15,databases
6,use href to train data and predict clas,16,machine-learning
7,use href to train data and predict clas,16,bigdata
8,use href to train data and predict clas,16,libsvm
9,lots of people use the term em big data em in ...,19,bigdata


In [19]:
my_df.to_csv('tag_and_post.csv',encoding='utf-8')

#### Build graph in neo4j

In [7]:
from py2neo import Graph, Node, Relationship

graph = Graph(password="password")

In [8]:
query = '''
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS FROM "file:////tag_and_post.csv" as row
MERGE (s:TAG{tag:row.tag})
MERGE (t:CONTENT{text:row.text})
CREATE (s)-[:TAGS]->(t)
'''

In [9]:
# run query
data = graph.run(query)
print(data)

#### all posts with tag 'octave'

In [ ]:
# MATCH p=(t:TAG)-[]->() WHERE t.tag = "octave" RETURN p LIMIT 25

![title](images/tag_octave.png)

#### all shortest path between 'java' and 'python' with some of the edge nodes tags shown

In [ ]:
# MATCH path = allShortestPaths( (t:TAG {tag:"java"})-[*]-(u:TAG {tag:"python"})) RETURN path LIMIT 10;

![title](images/python_java_shortpath.png)

 #### all shortest path between 'education' and 'java'

![title](images/java_education_shortpath.png)

 #### shortest path between 'activation' and 'xboost' with some of the edge nodes tags shown

![title](images/xboost_activation_shortpath.png)

#### Create edge list - node2vec

In [34]:
from neo4j import GraphDatabase
import csv

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

print('Create the edge list')
with driver.session() as session, open("graph/stack_data.edgelist", "w") as edges_file:
    result = session.run("""\
    MATCH (t:TAG)--(other)
    RETURN id(t) AS source, id(other) AS target
    """)

    writer = csv.writer(edges_file, delimiter=" ")

    for row in result:
        writer.writerow([row["source"], row["target"]])

Create the edge list


#### node2vec

In [35]:
%%bash
./node2vec -i:graph/movies.edgelist -o:emb/movies.emb -l:80 -d:100 -p:0.3 -dr -v

bash: line 1: ./node2vec: No such file or directory


CalledProcessError: Command 'b'./node2vec -i:graph/movies.edgelist -o:emb/movies.emb -l:80 -d:100 -p:0.3 -dr -v\n'' returned non-zero exit status 127.